In [ ]:
pip install pytesseract

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

In [ ]:
from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
path = glob("/content/drive/MyDrive/plates/annotations/*.xml")
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:
 info = xet.parse(filename)
 root = info.getroot()
 member_object = root.find('object')
 labels_info = member_object.find('bndbox')
 xmin = int(labels_info.find('xmin').text)
 xmax = int(labels_info.find('xmax').text)
 ymin = int(labels_info.find('ymin').text)
 ymax = int(labels_info.find('ymax').text)
 labels_dict['filepath'].append(filename)
 labels_dict['xmin'].append(xmin)
 labels_dict['xmax'].append(xmax)
 labels_dict['ymin'].append(ymin)
 labels_dict['ymax'].append(ymax)

In [ ]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/content/drive/MyDrive/plates/annotations/Cars...,115,278,116,153
1,/content/drive/MyDrive/plates/annotations/Cars...,230,248,129,134
2,/content/drive/MyDrive/plates/annotations/Cars...,158,277,216,248
3,/content/drive/MyDrive/plates/annotations/Cars...,152,206,147,159
4,/content/drive/MyDrive/plates/annotations/Cars...,141,311,113,164


In [ ]:
filename = df['filepath'][0]
def getFilename(filename):
 filename_image = xet.parse(filename).getroot().find('filename').text
 filepath_image = os.path.join('/content/drive/MyDrive/plates/images',filename_image)
 return filepath_image
getFilename(filename)

'/content/drive/MyDrive/plates/images/Cars109.png'

In [ ]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]#random chec

['/content/drive/MyDrive/plates/images/Cars109.png',
 '/content/drive/MyDrive/plates/images/Cars103.png',
 '/content/drive/MyDrive/plates/images/Cars108.png',
 '/content/drive/MyDrive/plates/images/Cars105.png',
 '/content/drive/MyDrive/plates/images/Cars107.png',
 '/content/drive/MyDrive/plates/images/Cars102.png',
 '/content/drive/MyDrive/plates/images/Cars100.png',
 '/content/drive/MyDrive/plates/images/Cars1.png',
 '/content/drive/MyDrive/plates/images/Cars106.png',
 '/content/drive/MyDrive/plates/images/Cars104.png']

In [ ]:
file_path = image_path[87] #path of our image N2.jpeg
img = cv2.imread(file_path) #read the image
# xmin-1804/ymin-1734/xmax-2493/ymax-1882
img = io.imread(file_path) #Read the image
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure8 - N2.jpeg with bounding box')
fig.add_shape(type='rect',x0=1804, x1=2493, y0=1734, y1=1882, xref='x',
yref='y',line_color='cyan')

In [ ]:
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
 image = image_path[ind]
 img_arr = cv2.imread(image)
 h,w,d = img_arr.shape
 # Prepprocesing
 load_image = load_img(image,target_size=(224,224))
 load_image_arr = img_to_array(load_image)
 norm_load_image_arr = load_image_arr/255.0 # Normalization
 # Normalization to labels
 xmin,xmax,ymin,ymax = labels[ind]
 nxmin,nxmax = xmin/w,xmax/w
 nymin,nymax = ymin/h,ymax/h
 label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
 # Append
 data.append(norm_load_image_arr)
 output.append(label_norm)

In [ ]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)
# Split the data into training and testing set using sklearn.
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

In [ ]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

219055592/219055592 [==============================] - 1s 0us/step


In [ ]:
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [ ]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),metrics=['accuracy'])

In [ ]:
tfb = TensorBoard('object_detection')
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=20,
 validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/20
35/35 [==============================] - 428s 12s/step - loss: 0.0163 - accuracy: 0.6503 - val_loss: 0.0234 - val_accuracy: 0.7471
Epoch 2/20
35/35 [==============================] - 381s 11s/step - loss: 0.0074 - accuracy: 0.7746 - val_loss: 0.0179 - val_accuracy: 0.7011
Epoch 3/20
35/35 [==============================] - 381s 11s/step - loss: 0.0060 - accuracy: 0.8497 - val_loss: 0.0198 - val_accuracy: 0.7471
Epoch 4/20
35/35 [==============================] - 393s 11s/step - loss: 0.0059 - accuracy: 0.8179 - val_loss: 0.0143 - val_accuracy: 0.7701
Epoch 5/20
35/35 [==============================] - 386s 11s/step - loss: 0.0042 - accuracy: 0.8410 - val_loss: 0.0123 - val_accuracy: 0.8276
Epoch 6/20
35/35 [==============================] - 405s 12s/step - loss: 0.0041 - accuracy: 0.8815 - val_loss: 0.0168 - val_accuracy: 0.7471
Epoch 7/20
35/35 [==============================] - 407s 12s/step - loss: 0.0040 - accuracy: 0.8786 - val_loss: 0.0159 - val_accuracy: 0.8046
Epoch 

In [ ]:
model.save('/content/drive/MyDrive/object_detection.h5')